In [ ]:
%load_ext autoreload

In [ ]:

from lavague.core import  WorldModel, ActionEngine
from lavague.core.agents import WebAgent
from lavague.drivers.selenium import SeleniumDriver
import os
from PIL import Image
import base64
from io import BytesIO
import pandas as pd


### Load test cases

In [ ]:
def find_and_parse_feature_file(directory):
    # List all files in the directory
    files = os.listdir(directory)
    
    # Find the first .feature file
    feature_file = None
    for file in files:
        if file.endswith('.feature'):
            feature_file = file
            break
    
    # If no .feature file is found, raise an error
    if not feature_file:
        raise FileNotFoundError("No .feature file found in the directory.")
    
    # Construct the full file path
    file_path = os.path.join(directory, feature_file)

    # Extract the file name without the extension
    file_name = os.path.splitext(feature_file)[0]

    # Read the contents of the file
    with open(file_path, 'r') as file:
        file_contents = file.read()

    return file_name, file_contents

# Example usage
directory = 'tests'  # Replace with your actual directory path

try:
    feature_file_name, test_case = find_and_parse_feature_file(directory)
    print(f"Feature Name: {feature_file_name}")
    print(f"Feature Contents:\n{test_case}")
except FileNotFoundError as e:
    print(e)


### Init and run LaVague

In [ ]:
selenium_driver = SeleniumDriver(headless=False)
world_model = WorldModel()
action_engine = ActionEngine(selenium_driver)
agent = WebAgent(world_model, action_engine)

URL = "https://www.saucedemo.com/v1/inventory.html"
OBJECTIVE = f"Run this test case: \n\n{test_case}"

In [ ]:
agent.get(URL)

In [ ]:
agent.run(OBJECTIVE, display=False)

### Parse logs

In [ ]:
logs = agent.logger.return_pandas()
# logs = pd.read_csv("logs_29052023-1157.csv")
logs.head()

In [ ]:
def get_latest_screenshot_path(directory):
    # List all files in the directory
    files = os.listdir(directory)
    
    # Get the full path of the files
    full_paths = [os.path.join(directory, f) for f in files]
    
    # Find the most recently modified file
    latest_file = max(full_paths, key=os.path.getmtime)
    
    return latest_file

def pil_image_to_base64(image_path):
    # Open the image file
    with Image.open(image_path) as img:
        # Convert image to BytesIO object
        # img.show()
        buffered = BytesIO()
        img.save(buffered, format="PNG")
        # Encode the BytesIO object to base64
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str


In [ ]:
# extract info from logs
# all the code
full_code = "\n".join(logs['code'].dropna())

# HTML from last step
last_step = logs.iloc[-1]
last_html = last_step['html']

# get nodes from the HTML
nodes = action_engine.get_nodes(f"We have ran the test case, generate a pytest-bdd assert statement.\n\ntest case:\n{test_case}")
last_screenshot_path = get_latest_screenshot_path(last_step["screenshots_path"])
b64_img = pil_image_to_base64(last_screenshot_path)

### Generate the pytest-bdd file

In [ ]:
EXAMPLES = """
import pytest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pytest_bdd import scenarios, given, when, then

# Constants
BASE_URL = 'https://form.jotform.com/241472287797370'

# Scenarios
scenarios('test_form_submission.feature')

# Fixtures
@pytest.fixture
def browser():
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(BASE_URL)
    yield driver
    driver.quit()

# Steps
@given('I am on the job application page')
def i_am_on_the_job_application_page(browser):
    pass

@when('I enter "John" in the "First Name" field')
def i_enter_first_name(browser):
    first_name_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[2]/div/div/span[1]/input")
    first_name_field.send_keys("John")

@when('I enter "Doe" in the "Last Name" field')
def i_enter_last_name(browser):
    last_name_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[2]/div/div/span[2]/input")
    last_name_field.send_keys("Doe")

@when('I enter "john.doe@example.com" in the "Email Address" field')
def i_enter_email_address(browser):
    email_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[3]/div/span/input")
    email_field.send_keys("john.doe@example.com")

@when('I enter "(123) 456-7890" in the "Phone Number" field')
def i_enter_phone_number(browser):
    phone_number_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[4]/div/span/input")
    phone_number_field.send_keys("(123) 456-7890")

@when('I leave the "Cover Letter" field empty')
def i_leave_cover_letter_empty():
    # No action needed as the field should remain empty
    pass

@when('I click the "Apply" button')
def i_click_apply_button(browser):
    apply_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/form/div[1]/ul/li[6]/div/div/button"))
    )
    browser.execute_script("arguments[0].scrollIntoView(true);", apply_button)
    apply_button.click()

@then('I should see an error message for the "Cover Letter" field')
def i_should_see_error_message(browser):
    try:
        error_message = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[5]/div/div/span")
        assert error_message.is_displayed()
    except Exception as e:
        pytest.fail(f"Error message not displayed: {e}")
"""
SYSTEM_PROMPT = """
You are an expert in software testing frameworks and python code generation. 
Your task is to read Gherkin tests, existing Selenium code, HTML and Screenshots of pages to generate an assert statement that will check only the last condition of the test case. Then you package everything in a pytest-bdd file.
Use descriptive function names, use execute_script if you think a click could result in an ElementClickInterceptedException. Code has to contain fixtures, scenario, gherkin style definitions, etc. 
Always use try-except blocks to catch exceptions and raise pytest.fail when checking the assert condition step when needed, name the scenario accordingly, follow good practices.
Never generate asserts for 'Given', 'When' or 'And' steps, only generate asserts for 'Then' steps.
Always re-use the selectors of the provided already executed code since we know it targets elements properly, never create new XPATH selector except to generate the final assert statement.
Only output code in all your responses. 
"""
USER_PROMPT = f"""
Generate a valid pytest-bdd file with the following inputs:

Base url:{URL}\n
Feature file name: {feature_file_name}\n
Test case:{test_case}\n
Already executed code:\n{full_code}\n
selected html of the last page:{nodes}

here are some examples for reference for testing a form:\n\n{EXAMPLES}
"""
print(USER_PROMPT)

In [ ]:
# convert the selenium code into a pytest-bdd file
from openai import OpenAI
def generate_pytest_file():
    print(test_case)
    api_key = "sk-..."
    client = OpenAI(api_key=api_key)
    # Define the code modification prompt
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system", 
                "content": SYSTEM_PROMPT
            },
            {
                "role": "user", 
                "content": [
                    {"type": "text", "text": USER_PROMPT},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{b64_img}"
}}
                ]
            }
        ]
    )

    return completion.choices[0].message.content


### Generate the code, cleanup and write directly to tests

In [ ]:
content = generate_pytest_file()

In [ ]:

pytest_code = content.strip("```python")
pytest_code = pytest_code.strip("```")
pytest_code = pytest_code.strip("```\n")

# TODO: ADD BETTER CODE CLEANUP/STRIPPING


In [ ]:

with open("tests/test_case.py", "w") as f:
    f.write(pytest_code)

### Run tests

In [ ]:
!pytest tests